In [ ]:
from transformers import AutoModelForCausalLM
from PIL import Image

import torch

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

device = "auto"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(r"/data2/pengziyang/huggingface/Ovis2-34B",
                                             torch_dtype=torch.bfloat16,
                                             multimodal_max_length=32768,
                                             trust_remote_code=True,
                                             device_map=device)
text_tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

In [ ]:
# cot_suffix = "Provide a step-by-step solution to every problems, and conclude with 'the answer is' followed by the final solution one by one."
# cot_suffix = "Provide a step-by-step solution to the problem, and conclude with 'the answer is' followed by the final solution."
cot_suffix = "请使用中文进行回答；对属于人和车的目标必须先分析他们的朝向，同时考虑目标在图中的位置与其到镜头的距离，提供解决问题的详细步骤，并在回答的末尾用‘答案是...’给出最终答案。"
# cot_suffix = "请使用中文进行回答；对属于人和车的目标必须先分析他们的朝向，同时考虑目标在图中的位置与其到镜头的距离，提供解决问题的详细步骤。"
# cot_suffix = "Please provide the bounding box coordinate range from of the region this sentence describes."
# image_path = r"/data2/pengziyang/data/VQA_SA/images/22_2_111_0000018_250205.jpg"
# image_path = r"./sln/VQA_SA/images/19_4_101_0000034_240516.jpg"
image_path = r"./sln/VQA_SA/images/12_0_101_0000020_250222.jpg"
# image_path = r"./sln/VQA_SA/images/26_4_104_0000036_250219.jpg"
images = [Image.open(image_path)]
# images = []
max_partition = 9
# history = """
# Here are some questions you answered before:

# Excuse me, which of the trash can in the picture and the child in white is closer to the photographer? Please choose from the trash can and the child in white
# Output:
# To determine which is closer to the photographer, the trash can or the child in white, we need to analyze their positions in the image. The child in white is walking on the pathway, while the trash can is located to the right side of the image. By comparing their relative distances, it is clear that the child in white is closer to the photographer than the trash can.

# Thus, the answer is the child in white.

# From the perspective of the little girl in white, in which direction is the trash can located? Please answer from the left, right, front, back, left front, right front, left back, and right back

# Output:
# To determine the direction of the trash can from the perspective of the little girl in white, we first locate the girl in the image. She is walking on the pathway, holding hands with an adult. Next, we identify the trash can, which is to the right side of the image. From the girl's perspective, the trash can is to her right. Therefore, the trash can is located to her right front.

# Thus, the answer is right front.

# """
# text = """他们中的哪一个更高？请回答水杯或者垃圾桶"""
# text = """从摄像角度看，路障在黑色电动车的哪个方向？请从前方、后方、左方、右方、上方、下方里选择一个方向

# 路障 位于 [731, 965, 780, 1264] 深度为 3.8
# 黑色电动车 位于 [839, 954, 1086, 1283] 深度为 3.8

# 已经有两个回答：
# 1、要从摄像头的视角确定路障相对于黑色电动车的方向，首先需要在图像中定位路障和黑色电动车。黑色电动车在远处可见，路障位于汽车的右侧。由于摄像头正对着场景，路障的位置应在黑色电动车的右侧。\n\n因此，答案是正确的。
# 2、首先，观察图像中路障和黑色电动车的位置。路障位于图像的右侧，而黑色电动车位于路障的右侧。根据深度信息，路障和黑色电动车的深度值均为3.8，说明它们距离镜头的距离相同。因此，从摄像机的视角来看，路障位于黑色电动车的左方。\n\n答案是左方。

# 请根据图像内容，从两种回答中整理出新的正确回答，请务必确保最后输出的答案在问题给出的答案选项中。
# """
# text = """汽车在戴头盔骑电动车的人的哪个方向？请从前方、后方、左方、右方、上方、下方、左前方、右前方、左后方、右后方中选择

# 汽车 位于 [563, 964, 710, 1071] 深度为 17.8
# 戴头盔骑电动车的人 位于 [703, 964, 801, 1125] 深度为 10.5

# 已经有两个回答：
# 1、若要确定汽车相对于戴头盔骑电动自行车的人的方向，首先需要在图像中定位这两个人。电动自行车骑行者位于图像的右侧，而汽车则位于他们的前方，稍偏左。通过分析他们的位置，我们可以得出结论：相对于骑电动自行车的人来说，汽车位于左前方。\n\n因此，正确答案是左前方。
# 2、为了确定汽车相对于戴头盔骑电动车的人的位置，我们首先分析图像中两个目标的相对位置和方向。汽车位于图像的中间偏左位置，而戴头盔骑电动车的人位于图像的右下方。根据深度信息，汽车的深度值为17.8，而戴头盔骑电动车的人的深度值为10.5，说明汽车更远离镜头。结合图像中两者的相对位置，汽车位于戴头盔骑电动车的人的左后方。\n\n因此，答案是左后方。

# 请根据图像内容，从两种回答中整理出新的正确回答，请务必确保最后输出的答案在问题给出的答案选项中。
# """
text = """图中黄色标识牌在撑伞的人的哪个方向？请从前方、后方、左方、右方、上方、下方、左前方、右前方、左后方、右后方中选择

黄色标识牌 位于 [416, 365, 440, 437] 深度为 15.7
撑伞的人 位于 [1260, 543, 1417, 908] 深度为 3.1 
撑伞的人朝向镜头方向的正左方

已经有两个回答：
1、要确定拿伞的人所指的黄色指示牌的方向，我们需要分析图像。拿伞的人位于图像的右侧。黄色指示牌位于拿伞的人的上方和左侧。因此，黄色指示牌指向拿伞的人的左前方。\n\n所以，正确答案是左前方。
2、首先，观察图像中撑伞的人的位置，位于图像的右侧。黄色标识牌位于图像的左侧，靠近道路的上方。根据图像中的位置和深度信息，黄色标识牌在撑伞的人的左后方。因此，黄色标识牌相对于撑伞的人的位置是左后方。\n\n答案是左后方。

请根据图像内容，从两种回答中整理出新的正确回答，请务必确保最后输出的答案在问题给出的答案选项中。
"""
# text = """以图中人的视角来看，请问铲子在他的哪个方位？请从前面或后面中选择一个。
# 图中人[430, 533, 717, 1049](深度 1.4)
# 铲子[163, 1175, 284, 1249](深度 0.3)"""
# text = """图中人 位于 [430, 533, 717, 1049] 深度 1.4 面向镜头视角的正左方
# 铲子 位于[163, 1175, 284, 1249] 深度 0.3
# 以图中人的视角来看，请问铲子在他的哪个方位？请从前面、后面、左侧、右侧中选择一个。"""
# text = """The red and white bucket on the far right of the center of the road near the junction"""
# query = f'{history}<image>\n{text}\n{cot_suffix}'
query = f'<image>\n{cot_suffix}: {text}'
# query = f'{cot_suffix}: {text}'

In [ ]:
prompt, input_ids, pixel_values = model.preprocess_inputs(query, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

In [ ]:
with torch.inference_mode():
    gen_kwargs = dict(
        max_new_tokens=1024,
        do_sample=False,
        top_p=None,
        top_k=None,
        temperature=None,
        repetition_penalty=None,
        eos_token_id=model.generation_config.eos_token_id,
        pad_token_id=text_tokenizer.pad_token_id,
        use_cache=True
    )
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = text_tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f'Output:\n{output}')

In [ ]:
# import cv2 as cv
from Portable import ipy
# import re

# perc_pattern = r"\[(\d\.\d+)[^\d\.]+(\d\.\d+)[^\d\.]+(\d\.\d+)[^\d\.]+(\d\.\d+)\]"
# coords = [float(e) for e in re.findall(perc_pattern, output)[0]]

# canvas = cv.imread(image_path, cv.IMREAD_COLOR)
# H, W, C = canvas.shape
# coords = [round(e * W if i % 2 == 0 else e * H) for i, e in enumerate(coords)]
# canvas = cv.rectangle(canvas, coords[:2], coords[2:], (0, 0, 255), 2)
# print(text)
# ipy.display_np_images(canvas, resize=0.4, bgr2rgb=True)

ipy.display_image_files(image_path)